In [1]:
using Distributions
using StatsBase
using CSV
using DataFrames
using HypothesisTests
using Plots
using GLM

In [2]:
test1 = CSV.read("ProjectData_1_point_0.csv");

In [3]:
test1

,Age,WCC,CRP,Treatment,Result
,Int64,Float64,Int64,String,String
1,39,9.3,10,A,Worse
2,51,14.6,50,A,Worse
3,75,7.1,50,A,Improved
4,59,12.0,10,A,Improved
5,62,13.6,60,A,Static
6,59,11.9,30,B,Static
7,31,10.6,10,B,Static
8,63,11.8,70,A,Worse
9,41,13.2,70,A,Worse


In [4]:
DataFrame(Group=rand(["A", "B"], 20), Variable1=randn(20), Variable2=rand(20))

,Group,Variable1,Variable2
,String,Float64,Float64
1,B,-0.395164,0.785108
2,A,-0.477911,0.94664
3,B,1.3248,0.683507
4,A,2.18945,0.655638
5,B,-1.8022,0.40504
6,B,-0.0415339,0.0649169
7,A,-0.988776,0.579329
8,B,0.452246,0.0049928
9,B,-1.23054,0.529847


In [5]:
view(test1,3,:)

,Age,WCC,CRP,Treatment,Result
,Int64,Float64,Int64,String,String
3,75,7.1,50,A,Improved


In [17]:
test1[[1:2;4:4;6:8;10:end],:]

,Age,WCC,CRP,Treatment,Result
,Int64,Float64,Int64,String,String
1,39,9.3,10,A,Worse
2,51,14.6,50,A,Worse
3,59,12.0,10,A,Improved
4,59,11.9,30,B,Static
5,31,10.6,10,B,Static
6,63,11.8,70,A,Worse
7,45,13.6,90,B,Worse
8,78,11.8,70,B,Improved
9,36,13.7,50,A,Improved


In [19]:
var(rand(Normal(80, 10), 200))

101.20293783633616

In [21]:
mean(randn(100))

-0.0520584551299503

In [ ]:
TDist()

In [ ]:
EqualVarianceTTest()